In [4]:
# Ignore warnings
import warnings
warnings.simplefilter(action='ignore')

# Prerequisites
#import sys
#!{sys.executable} -m pip install scikit-learn folium pygeohash catboost xgboost lightgbm

# Load Dependencies
import time
from sklearn.pipeline import Pipeline
import pickle
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
import xgboost as xgb

#import folium.plugins as plugins

from math import sqrt
from sklearn import metrics
from sklearn.svm import SVR
#from folium.plugins import HeatMap
#from IPython.display import display
from catboost import CatBoostRegressor
from scipy.stats import boxcox, zscore
from statsmodels.formula.api import ols
#from sklearn.feature_selection import RFE
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge, LinearRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, PolynomialFeatures

# Display
%matplotlib inline
sns.set(font_scale=1)
sns.set_style("white")


import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

In [5]:
train = pd.read_csv("https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/df-train_set.csv")
test = pd.read_csv('https://raw.githubusercontent.com/MathewJohn1986/Team-8-Predict/main/df-test_set.csv')

In [6]:
train=train[train['Commodities'].str.contains('APPLE GOLDEN DELICIOUS')]

In [7]:
index = test.index
index

RangeIndex(start=0, stop=685, step=1)

In [8]:
test.set_index('Index', inplace=True)

In [9]:
test

,Province,Container,Size_Grade,Weight_Kg,Commodities,Date,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand
Index,,,,,,,,,,,,
1,W.CAPE-BERGRIVER ETC,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-07-09,128.0,136.0,5008.0,38,456.0,0
2,W.CAPE-BERGRIVER ETC,M4183,1X,18.3,APPLE GOLDEN DELICIOUS,2020-01-20,220.0,220.0,1760.0,8,146.4,2
3,W.CAPE-BERGRIVER ETC,EC120,1S,12.0,APPLE GOLDEN DELICIOUS,2020-08-19,120.0,120.0,720.0,6,72.0,45
4,W.CAPE-BERGRIVER ETC,M4183,1M,18.3,APPLE GOLDEN DELICIOUS,2020-05-06,160.0,160.0,160.0,1,18.3,8
5,W.CAPE-BERGRIVER ETC,M4183,1L,18.3,APPLE GOLDEN DELICIOUS,2020-05-04,140.0,160.0,14140.0,100,1830.0,19
...,...,...,...,...,...,...,...,...,...,...,...,...
681,W.CAPE-BERGRIVER ETC,JE090,2S,9.0,APPLE GOLDEN DELICIOUS,2020-05-26,30.0,30.0,1320.0,44,396.0,856
682,CAPE,EC120,1M,12.0,APPLE GOLDEN DELICIOUS,2020-01-10,50.0,132.0,6290.0,52,624.0,785
683,W.CAPE-BERGRIVER ETC,JE090,2X,9.0,APPLE GOLDEN DELICIOUS,2020-06-03,55.0,55.0,220.0,4,36.0,170


In [10]:
train = pd.get_dummies(train, columns=['Province', 'Container', 'Size_Grade','Commodities'],drop_first = True)
test= pd.get_dummies(test, columns=['Province', 'Container', 'Size_Grade','Commodities'],drop_first = True)
train['avg_price_per_kg'] = train.pop('avg_price_per_kg')
train= train.drop('Date', axis=1)
test= test.drop('Date', axis=1)
X = train.iloc[:, :-1].values
y = train.iloc[:, -1].values
X_test = test.iloc[:,:].values


In [11]:
train

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Province_EASTERN CAPE,Province_NATAL,Province_ORANGE FREE STATE,...,Size_Grade_1M,Size_Grade_1S,Size_Grade_1U,Size_Grade_1X,Size_Grade_2L,Size_Grade_2M,Size_Grade_2S,Size_Grade_2U,Size_Grade_2X,avg_price_per_kg
1,18.3,150.0,170.0,51710.0,332,6075.6,822,0,0,0,...,0,0,0,0,0,0,0,0,0,8.51
7,11.0,50.0,50.0,16000.0,320,3520.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,4.55
24,9.0,55.0,55.0,990.0,18,162.0,1506,0,0,0,...,0,0,0,0,0,0,1,0,0,6.11
40,18.3,80.0,120.0,32020.0,388,7100.4,443,0,0,0,...,0,1,0,0,0,0,0,0,0,4.51
69,400.0,1800.0,1800.0,1800.0,1,400.0,2,1,0,0,...,0,1,0,0,0,0,0,0,0,4.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64276,12.0,96.0,96.0,1920.0,20,240.0,53,0,1,0,...,0,1,0,0,0,0,0,0,0,8.00
64291,18.3,130.0,130.0,7280.0,56,1024.8,74,0,0,1,...,0,0,0,1,0,0,0,0,0,7.10
64297,9.0,50.0,60.0,4540.0,89,801.0,793,0,0,0,...,0,0,0,0,0,1,0,0,0,5.67
64304,11.0,30.0,40.0,2140.0,70,770.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,2.78


In [12]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

print("X_train:", X_train[0,0:11], "... Training")
print("y_train:", y_train[0])
print("X_val:", X_val[0,0:11], "... Validation")
print("y_val:", y_val[0])
print("X_test:", X_test[0,0:11], "... Testing")

X_train: [9.000e+00 5.500e+01 6.500e+01 1.275e+03 2.100e+01 1.890e+02 1.300e+01
 1.000e+00 0.000e+00 0.000e+00 0.000e+00] ... Training
y_train: 6.75
X_val: [1.20e+01 4.00e+01 4.00e+01 2.04e+03 5.10e+01 6.12e+02 0.00e+00 0.00e+00
 1.00e+00 0.00e+00 0.00e+00] ... Validation
y_val: 3.33
X_test: [  12.  128.  136. 5008.   38.  456.    0.    0.    0.    0.    0.] ... Testing


In [13]:
# Make new copies for scaling
X_train_scaled = X_train.copy()
X_val_scaled = X_val.copy()
X_test_scaled = X_test.copy()

# Apply scaler to numeric features
sc = StandardScaler()
X_train_scaled[:,4:] = sc.fit_transform(X_train[:,4:])
X_val_scaled[:,4:] = sc.transform(X_val[:,4:])
X_test_scaled[:,4:] = sc.transform(X_test[:,4:])

In [14]:
# Convert to dataframes
all_columns = ['Weight_Kg', 'Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold',
       'Total_Kg_Sold', 'Stock_On_Hand', 'Province_EASTERN CAPE',
       'Province_NATAL', 'Province_ORANGE FREE STATE', 'Province_TRANSVAAL',
       'Province_W.CAPE-BERGRIVER ETC', 'Province_WEST COAST',
       'Container_DT063', 'Container_EC120', 'Container_EF120',
       'Container_EG140', 'Container_IA400', 'Container_JE090',
       'Container_JG110', 'Container_M4183', 'Container_M6125',
       'Container_M9125', 'Size_Grade_1M', 'Size_Grade_1S', 'Size_Grade_1U',
       'Size_Grade_1X', 'Size_Grade_2L', 'Size_Grade_2M', 'Size_Grade_2S',
       'Size_Grade_2U', 'Size_Grade_2X', ]
# Not scaled
X_train_df = pd.DataFrame(X_train,columns=all_columns)
X_val_df = pd.DataFrame(X_val,columns=all_columns)
X_test_df = pd.DataFrame(X_test,columns=all_columns)
X_all_df = pd.DataFrame(X,columns=all_columns)
# Scaled
X_train_scaled_df = pd.DataFrame(X_train_scaled,columns=all_columns)
X_val_scaled_df = pd.DataFrame(X_val_scaled,columns=all_columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled,columns=all_columns)
# Convert to numeric
X_train_scaled_df = X_train_scaled_df.apply(pd.to_numeric)
X_val_scaled_df = X_val_scaled_df.apply(pd.to_numeric)
X_test_scaled_df = X_test_scaled_df.apply(pd.to_numeric)
X_train_df = X_train_df.apply(pd.to_numeric)
X_val_df = X_val_df.apply(pd.to_numeric)
X_test_df = X_test_df.apply(pd.to_numeric)
X_all_df = X_all_df.apply(pd.to_numeric)

X_train_scaled_df.head()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Province_EASTERN CAPE,Province_NATAL,Province_ORANGE FREE STATE,...,Container_M9125,Size_Grade_1M,Size_Grade_1S,Size_Grade_1U,Size_Grade_1X,Size_Grade_2L,Size_Grade_2M,Size_Grade_2S,Size_Grade_2U,Size_Grade_2X
0,9.0,55.0,65.0,1275.0,-0.538134,-0.459554,-0.548228,2.918611,-0.242947,-0.067116,...,-0.043881,-0.439801,-0.621549,-0.025318,-0.310174,-0.258816,2.744095,-0.422035,-0.088017,-0.119562
1,400.0,1200.0,1200.0,1200.0,-0.607961,-0.425434,-0.535630,-0.342629,-0.242947,-0.067116,...,-0.043881,-0.439801,1.608883,-0.025318,-0.310174,-0.258816,-0.364419,-0.422035,-0.088017,-0.119562
2,12.0,80.0,104.0,4008.0,-0.461324,-0.406676,-0.469839,-0.342629,-0.242947,-0.067116,...,-0.043881,2.273755,-0.621549,-0.025318,-0.310174,-0.258816,-0.364419,-0.422035,-0.088017,-0.119562
3,12.0,84.0,96.0,1584.0,-0.548608,-0.455188,-0.185679,-0.342629,-0.242947,-0.067116,...,-0.043881,-0.439801,1.608883,-0.025318,-0.310174,-0.258816,-0.364419,-0.422035,-0.088017,-0.119562
4,12.0,120.0,128.0,4704.0,-0.478781,-0.416379,-0.499235,-0.342629,-0.242947,-0.067116,...,-0.043881,-0.439801,1.608883,-0.025318,-0.310174,-0.258816,-0.364419,-0.422035,-0.088017,-0.119562


In [15]:
# Final features used in model
features =  ['Weight_Kg', 'Low_Price', 'High_Price', 'Sales_Total', 'Total_Qty_Sold',
       'Total_Kg_Sold', 'Stock_On_Hand', 'Province_EASTERN CAPE',
       'Province_NATAL', 'Province_ORANGE FREE STATE', 'Province_TRANSVAAL',
       'Province_W.CAPE-BERGRIVER ETC', 'Province_WEST COAST',
       'Container_DT063', 'Container_EC120', 'Container_EF120',
       'Container_EG140', 'Container_IA400', 'Container_JE090',
       'Container_JG110', 'Container_M4183', 'Container_M6125',
       'Container_M9125', 'Size_Grade_1M', 'Size_Grade_1S', 'Size_Grade_1U',
       'Size_Grade_1X', 'Size_Grade_2L', 'Size_Grade_2M', 'Size_Grade_2S',
       'Size_Grade_2U', 'Size_Grade_2X', ]
X_all = pd.DataFrame(X_all_df.loc[:,features],columns=features).values
X_train = pd.DataFrame(X_train_df.loc[:,features],columns=features).values
X_val = pd.DataFrame(X_val_df.loc[:,features],columns=features).values
X_test = pd.DataFrame(X_test_df.loc[:,features],columns=features).values
X_train_scaled = pd.DataFrame(X_train_scaled_df.loc[:,features],columns=features).values
X_val_scaled = pd.DataFrame(X_val_scaled_df.loc[:,features],columns=features).values

In [16]:
modelstart= time.time()
cb_model = CatBoostRegressor(iterations=700,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 0,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
cb_model.fit(X_train, y_train, eval_set=(X_val,y_val), use_best_model=True,verbose=True)
y_pred_val = cb_model.predict(X_val)
cb_model_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred_val))
cb_model_r2 = metrics.r2_score(y_val, y_pred_val)
print('RMSE:', cb_model_rmse)
print('R-squared:', cb_model_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)))

0:	learn: 2.2143797	test: 2.2377650	best: 2.2377650 (0)	total: 304ms	remaining: 3m 32s
75:	learn: 1.0179910	test: 1.0741032	best: 1.0741032 (75)	total: 7.16s	remaining: 58.8s
150:	learn: 0.6808264	test: 0.7679329	best: 0.7679329 (150)	total: 14.3s	remaining: 51.9s
225:	learn: 0.5447580	test: 0.6751722	best: 0.6751722 (225)	total: 21.4s	remaining: 44.9s
300:	learn: 0.4607775	test: 0.6356531	best: 0.6356531 (300)	total: 28.8s	remaining: 38.2s
375:	learn: 0.4003074	test: 0.6082941	best: 0.6082941 (375)	total: 37.8s	remaining: 32.6s
450:	learn: 0.3536048	test: 0.5904075	best: 0.5904075 (450)	total: 45.7s	remaining: 25.2s
525:	learn: 0.3184649	test: 0.5780496	best: 0.5780496 (525)	total: 54.9s	remaining: 18.2s
600:	learn: 0.2871319	test: 0.5695803	best: 0.5695407 (599)	total: 1m 3s	remaining: 10.5s
675:	learn: 0.2620431	test: 0.5625121	best: 0.5625121 (675)	total: 1m 11s	remaining: 2.54s
699:	learn: 0.2544815	test: 0.5606793	best: 0.5606793 (699)	total: 1m 13s	remaining: 0us

bestTest = 0.5

In [30]:
# Make submission file
final_model = CatBoostRegressor(iterations=700,
                             learning_rate=0.02,
                             depth=12,
                             eval_metric='RMSE',
                             random_seed = 0,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 75,
                             od_wait=100)
final_model.fit(X_all,y)
y_test = final_model.predict(X_test)
results = pd.DataFrame({"Index":index,"avg_price_per_kg": y_test})
results.to_csv("catbooster5", index=False)

0:	learn: 2.2193222	total: 152ms	remaining: 1m 46s
75:	learn: 0.9957899	total: 7.71s	remaining: 1m 3s
150:	learn: 0.6666034	total: 14.7s	remaining: 53.4s
225:	learn: 0.5335207	total: 22.1s	remaining: 46.3s
300:	learn: 0.4606947	total: 30s	remaining: 39.7s
375:	learn: 0.4084765	total: 38.1s	remaining: 32.8s
450:	learn: 0.3717035	total: 45.8s	remaining: 25.3s
525:	learn: 0.3416191	total: 53.4s	remaining: 17.7s
600:	learn: 0.3114730	total: 1m 1s	remaining: 10.1s
675:	learn: 0.2819329	total: 1m 9s	remaining: 2.46s
699:	learn: 0.2741889	total: 1m 11s	remaining: 0us


In [31]:
modelstart= time.time()
final_model = xgb.XGBRegressor(max_features= 5, min_samples_leaf=1, min_samples_split= 2, n_estimators= 300)
final_model.fit(X_train,y_train)
y_pred_val_final = final_model.predict(X_val)
y_pred_test_final = final_model.predict(X_test)
final_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred_val_final))
final_r2 = metrics.r2_score(y_val, y_pred_val_final)
print("Testing: LightGBM")
print('RMSE:', final_rmse)
print('R-squared:', final_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)/60))

[21:46:06] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { max_features, min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Testing: LightGBM
RMSE: 0.48136227570143847
R-squared: 0.9547681171583852
Model Runtime: 0.02 seconds


In [32]:
# Make submission file
final_model =  xgb.XGBRegressor(max_features= 5, min_samples_leaf=1, min_samples_split= 2, n_estimators= 300)
final_model.fit(X_all,y)
y_test = final_model.predict(X_test)
results = pd.DataFrame({"Index":index,"avg_price_per_kg": y_test})
results.to_csv("xgbooster5", index=False)

[21:46:51] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_1619728204606/work/src/learner.cc:541: 
Parameters: { max_features, min_samples_leaf, min_samples_split } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [69]:
modelstart= time.time()
xg_reg = GradientBoostingRegressor()
xg_reg.fit(X_train,y_train)
y_pred = xg_reg.predict(X_val)
xg_rmse = sqrt(metrics.mean_squared_error(y_val, y_pred))
xg_r2 = metrics.r2_score(y_val, y_pred)

print("Testing: XGBoost")
print('RMSE:', xg_rmse)
print('R-squared:', xg_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)))

Testing: XGBoost
RMSE: 0.5716035988427546
R-squared: 0.9362191251656652
Model Runtime: 0.28 seconds


In [30]:
RF = RandomForestRegressor(n_estimators=100, max_depth=5)
RF.fit(X_train,y_train)
# Get predictions
y_pred = RF.predict(X_val)
RF_r2 = metrics.r2_score(y_val,y_pred)
print("Testing: RandomForest")
print("RMSE:",np.sqrt(mean_squared_error(y_val,y_pred)))
print('R-squared:', RF_r2)
print("Model Runtime: %0.2f seconds"%((time.time() - modelstart)))

Testing: RandomForest
RMSE: 0.7974040328132999
R-squared: 0.8758755267740754
Model Runtime: 1214.47 seconds


In [34]:
# Create LASSO model object, setting alpha to 0.01
lasso = Lasso(alpha=0.01)
# Train the LASSO model
lasso.fit(X_train, y_train)

Lasso(alpha=0.01)

In [35]:
# Least squares using non-zero variables from LASSO
lm_subset = LinearRegression()
# Least squares using all predictors
lm_all = LinearRegression()
# Ridge using all predictors
ridge = Ridge()

lm_subset.fit(X_train, y_train)
lm_all.fit(X_train, y_train)
ridge.fit(X_train, y_train)
# Make training set predictions for each model
train_lm_subset = lm_subset.predict(X_train)
train_lm_all = lm_all.predict(X_train)
train_ridge = ridge.predict(X_train)
train_lasso = lasso.predict(X_train)

In [36]:
# Make test set predictions for each model
test_lm_subset = lm_subset.predict(X_val)
test_lm_all = lm_all.predict(X_val)
test_ridge = ridge.predict(X_val)
test_lasso = lasso.predict(X_val)

In [37]:
# Dictionary of results
results_dict = {'Training MSE':
                    {
                        "Least Squares, Subset": metrics.mean_squared_error(y_train, train_lm_subset),
                        "Least Squares, All": metrics.mean_squared_error(y_train, train_lm_all),
                        "Ridge": metrics.mean_squared_error(y_train, train_ridge),
                        "LASSO": metrics.mean_squared_error(y_train, train_lasso)
                    },
                    'Test MSE':
                    {
                        "Least Squares, Subset": metrics.mean_squared_error(y_val, test_lm_subset),
                        "Least Squares, All": metrics.mean_squared_error(y_val, test_lm_all),
                        "Ridge": metrics.mean_squared_error(y_val, test_ridge),
                        "LASSO": metrics.mean_squared_error(y_val, test_lasso)
                    }
                }
# Create dataframe from dictionary
results_df = pd.DataFrame(data=results_dict)
# View the results
results_df

,Training MSE,Test MSE
"Least Squares, Subset",1.950744,2.161358
"Least Squares, All",1.950744,2.161358
Ridge,1.955751,2.125344
LASSO,2.046642,2.161228
